20163228 남유선

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd

# Read Files

In [2]:
#
# Read Train File	
#
size_row    = 28    # height of the image
size_col    = 28    # width of the image

train_file  = "mnist_train.csv"

handle_file = open(train_file, "r")
train_data  = handle_file.readlines()
handle_file.close()

train_num   = len(train_data)

train_list  = np.zeros((train_num, size_row * size_col), dtype=float)
train_a     = np.zeros((train_num, size_row * size_col + 1), dtype=float)
train_label = np.zeros((10, train_num), dtype=int)

count = 0
label = 2
for line in train_data :
    line_data = line.split(',')
    
    for i in range (10) :
        if (line_data[0] == str(i)) :
            train_label[i, count] = 1
        else :
            train_label[i, count] = -1
    
    # Image Vector
    im_vector = np.asfarray(line_data[1:])
    train_list[count, :] = im_vector
    
    train_a[count, 0]    = 1
    train_a[count, 1:]   = im_vector
    count += 1

In [3]:
#
# Read Test File
#
test_file   = "mnist_test.csv"

handle_file = open(test_file, "r")
test_data   = handle_file.readlines()
handle_file.close()

test_num    = len(test_data)

test_list   = np.zeros((test_num, size_row * size_col), dtype=float)
test_a      = np.zeros((test_num, size_row * size_col + 1), dtype=float)
test_label = np.zeros((10, test_num), dtype=int)

count = 0
label = -2
for line in test_data :
    line_data = line.split(',')
    
    for i in range (10) :
        if (line_data[0] == str(i)) :
            test_label[i, count] = 1
        else :
            test_label[i, count] = -1
    
    im_vector = np.asfarray(line_data[1:])
    test_list[count, :] = im_vector
    
    test_a[count, 0]    = 1
    test_a[count, 1:]   = im_vector
    count += 1

# Calculate Pseudo Inverse

In [4]:
class LeastSquare() :
    
    def __init__(self, image, label) :
        self.image = image
        self.label = label
        self.n     = len(label)
        self.m     = len(image[0])
        
        
    def calculate_ata(self) :
        self.trans_image = self.image.T
        self.ata = np.dot(self.trans_image, self.image)
        
        
    def calculate_atb(self) :
        re_label = self.label.reshape(self.n, 1)
        self.atb = np.dot(self.trans_image, re_label)
        
        
    def calculate_theta(self) :
        mati_ata = np.linalg.pinv(self.ata)
        mat_atb = np.asmatrix(self.atb)
        self.aia = np.dot(mati_ata, mat_atb)
        
        
    def train(self) :
        self.calculate_ata()
        self.calculate_atb()
        
        self.calculate_theta()
        
        return self.aia

In [5]:
theta = np.zeros((10, size_row * size_col + 1), dtype=float)
for i in range (10) :
    ls = LeastSquare(train_a, train_label[i, :])
    setaa= ls.train()
    theta[i] = setaa.reshape(785)

In [6]:
def calculate_y(matrix_a, theta) :
    result   = np.dot(matrix_a, theta)
    y_result = np.zeros(len(matrix_a), dtype=int)
    
    for i in range (len(matrix_a)) :
        if (result[i] > 0) :
            y_result[i] = 1
        else :
            y_result[i] = -1
            
    return y_result

In [7]:
train_pred = np.zeros((10, train_num), dtype=float)
test_pred  = np.zeros((10, test_num), dtype=float)

for i in range (10) :
    train_pred[i] = calculate_y(train_a, theta[i])
    test_pred[i]  = calculate_y(test_a , theta[i])

# Calculate Accuracy

In [27]:
def calculate_accuracy(true, pred) :
    count = np.zeros((2, 2), dtype=float)
    accuracy = np.zeros((2, 2), dtype=float)
    
    for i in range (len(true)) :
        if ((true[i] == 1) and (pred[i] == 1)) :      # TP
            count[0][0] += 1
        elif ((true[i] == 1) and (pred[i] == -1)) :   # FN
            count[0][1] += 1
        elif ((true[i] == -1) and (pred[i] == 1)) :   # FP
            count[1][0] += 1
        elif ((true[i] == -1) and (pred[i] == -1)) :  # TN
            count[1][1] += 1
            
    accuracy[0][0] = count[0][0] / (count[0][0] + count[0][1])
    accuracy[0][1] = count[0][1] / (count[0][0] + count[0][1])
    accuracy[1][0] = count[1][0] / (count[1][0] + count[1][1])
    accuracy[1][1] = count[1][1] / (count[1][0] + count[1][1])

#     show_table(accuracy)
    tp = count[0][0] / (count[0][0] + count[0][1])
    er = (count[0][1] + count[1][0]) / len(true)
    
#     print("True Positive Rate : ", tp)
#     print("Error Rate         : ", er)
    return tp, er

In [28]:
def show_table(accuracy) :
    data = {"True Positive" : [accuracy[0][0], accuracy[1][0], accuracy[2][0], accuracy[3][0], accuracy[4][0], accuracy[5][0], accuracy[6][0], accuracy[7][0], accuracy[8][0], accuracy[9][0]], 
            "Error Rate"         : [accuracy[0][1], accuracy[1][1], accuracy[2][1], accuracy[3][1], accuracy[4][1], accuracy[5][1], accuracy[6][1], accuracy[7][1], accuracy[8][1], accuracy[9][1]]}
    df = pd.DataFrame(data, columns=["True Positive", "Error Rate"], index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    print(df)

In [29]:
train_accuracy = np.zeros((10, 2), dtype=float)
for i in range(10) :
#     print("\n<<", i, ">>")
    train_accuracy[i][0], train_accuracy[i][1] = calculate_accuracy(train_label[i], train_pred[i])
show_table(train_accuracy)

   True Positive  Error Rate
0       0.872531    0.015567
1       0.909522    0.019017
2       0.660121    0.037667
3       0.625999    0.042233
4       0.690688    0.033783
5       0.441063    0.055567
6       0.838459    0.021667
7       0.723703    0.033983
8       0.511366    0.051350
9       0.523281    0.055633


In [30]:
test_accuracy = np.zeros((10, 2), dtype=float)
for i in range(10) :
#     print("\n<<", i, ">>")
    test_accuracy[i][0], test_accuracy[i][1] = calculate_accuracy(test_label[i], test_pred[i])
show_table(test_accuracy)

   True Positive  Error Rate
0       0.883673      0.0157
1       0.911894      0.0166
2       0.624031      0.0416
3       0.648515      0.0398
4       0.697556      0.0335
5       0.461883      0.0536
6       0.805846      0.0260
7       0.714981      0.0354
8       0.517454      0.0510
9       0.555005      0.0520
